## Business Problem

Restaurants play a major role in the business, intellectual, social and artistic life of a society. To get into a restaurant business, one need to have a proper business plan and background analysis to seek investors attention. For a restaurant business to be in a good shape, customer ratings play a key role. In today's world, web and apps allow customers to efficiently express their views and feedback on the restaurants to a large audience. This is valuable as one can get in-depth information regarding a restaurant from other customers for a very low efforts and search costs. It is always better to have a thorough analysis of the market before getiing into the business.

We are assuming ourselves as a consulting firm who would provide valuable insights and analysis to the individuals who are planning to start a new restaurant in Banglore city. In the below analysis we have determined various factors that plays role in deciding the customer ratings for a restaurant. We also built predictive models that helps in determining a restaurants success(customer ratings) based on the business plan.

For this analysis, we have used an open source dataset scrapped from Zomato available in Kaggle.

In [2]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('PROJECT').getOrCreate()

In [5]:
#importing data to a pandas dataframe
import pandas as pd
df = pd.read_csv("/dbfs/FileStore/tables/zomato.csv")

In [6]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
# As reading data directly into spark dataframe is causing data mismatch for various rows, 
# so we have first imported using pandas and then converted it into spark dataframe.

In [7]:
df = spark.createDataFrame(df)

In [8]:
df.show(n=5)

+--------------------+--------------------+--------------------+------------+----------+-----+-----+--------------------+------------+-------------------+--------------------+--------------------+---------------------------+--------------------+---------+---------------+---------------+
 url| address| name|online_order|book_table| rate|votes| phone| location| rest_type| dish_liked| cuisines|approx_cost(for two people)| reviews_list|menu_item|listed_in(type)|listed_in(city)|
+--------------------+--------------------+--------------------+------------+----------+-----+-----+--------------------+------------+-------------------+--------------------+--------------------+---------------------------+--------------------+---------+---------------+---------------+
https://www.zomat...|942, 21st Main Ro...| Jalsa| Yes| Yes|4.1/5| 775|080 42297555
+91...|Banashankari| Casual Dining|Pasta, Lunch Buff...|North Indian, Mug...| 800|[('Rated 4.0', 'R...| []| Buffet| Banashankari|
https://www.zomat...|2nd Floor, 80 Fee...| Spice Elephant| Yes| No|4.1/5| 787| 080 41714161|Banashankari| Casual Dining|Momos, Lunch Buff...|Chinese, North In...| 800|[('Rated 4.0', 'R...| []| Buffet| Banashankari|
https://www.zomat...|1112, Next to KIM...| San Churro Cafe| Yes| No|3.8/5| 918| +91 9663487993|Banashankari|Cafe, Casual Dining|Churros, Cannello...|Cafe, Mexican, It...| 800|[('Rated 3.0', "R...| []| Buffet| Banashankari|
https://www.zomat...|1st Floor, Annaku...|Addhuri Udupi Bho...| No| No|3.7/5| 88| +91 9620009302|Banashankari| Quick Bites| Masala Dosa|South Indian, Nor...| 300|[('Rated 4.0', "R...| []| Buffet| Banashankari|
https://www.zomat...|10, 3rd Floor, La...| Grand Village| No| No|3.8/5| 166|+91 8026612447
+...|Basavanagudi| Casual Dining| Panipuri, Gol Gappe|North Indian, Raj...| 600|[('Rated 4.0', 'R...| []| Buffet| Banashankari|
+--------------------+--------------------+--------------------+------------+----------+-----+-----+--------------------+------------+-------------------+--------------------+--------------------+---------------------------+--------------------+---------+---------------+---------------+
only showing top 5 rows

In [9]:
df.printSchema()

root
-- url: string (nullable = true)
-- address: string (nullable = true)
-- name: string (nullable = true)
-- online_order: string (nullable = true)
-- book_table: string (nullable = true)
-- rate: string (nullable = true)
-- votes: long (nullable = true)
-- phone: string (nullable = true)
-- location: string (nullable = true)
-- rest_type: string (nullable = true)
-- dish_liked: string (nullable = true)
-- cuisines: string (nullable = true)
-- approx_cost(for two people): string (nullable = true)
-- reviews_list: string (nullable = true)
-- menu_item: string (nullable = true)
-- listed_in(type): string (nullable = true)
-- listed_in(city): string (nullable = true)

In [10]:
df.count()

Out[9]: 51717

In [11]:
df.dropna()

Out[10]: DataFrame[url: string, address: string, name: string, online_order: string, book_table: string, rate: string, votes: bigint, phone: string, location: string, rest_type: string, dish_liked: string, cuisines: string, approx_cost(for two people): string, reviews_list: string, menu_item: string, listed_in(type): string, listed_in(city): string]

In [12]:
df.columns

Out[11]: ['url',
 'address',
 'name',
 'online_order',
 'book_table',
 'rate',
 'votes',
 'phone',
 'location',
 'rest_type',
 'dish_liked',
 'cuisines',
 'approx_cost(for two people)',
 'reviews_list',
 'menu_item',
 'listed_in(type)',
 'listed_in(city)']

In [13]:
# Rating column has attributes in the fraction form (4/5,3/5..). We only need numerator part of it for the analysis.
# Thus we are splitting it accordingly and creating a new column named ratings in this case.
import pyspark.sql.functions as f

split_col = f.split(df['rate'], '/')
df = df.withColumn('ratings', split_col.getItem(0))

In [14]:
# As low number of votes indicates that the reviews are not authentic. 
# So, we are considering the data points for which the vote count is greater than 150.

df1=df.filter("votes > 150")

In [15]:
# Here we have included two new features to the existing data frame.
df1 = df1.withColumn('CuisinesCount', f.size(f.split(f.col('cuisines'), ',')))
df1 = df1.withColumn('dish_liked_Count', f.size(f.split(f.col('dish_liked'), ',')))

df1 = df1.withColumnRenamed('listed_in(city)','listed_in_city')
df1 = df1.withColumnRenamed( 'approx_cost(for two people)', 'approx_cost')
df1 = df1.withColumnRenamed('listed_in(type)','listed_in_type')

from pyspark.sql.types import IntegerType
df1 = df1.withColumn("approx_cost", df1["approx_cost"].cast(IntegerType()))
df1 = df1.withColumn("ratings", df1["ratings"].cast('float'))

df1.show(n=5)

+--------------------+--------------------+---------------+------------+----------+-----+-----+--------------------+------------+-------------------+--------------------+--------------------+-----------+--------------------+---------+--------------+--------------+-------+-------------+----------------+
 url| address| name|online_order|book_table| rate|votes| phone| location| rest_type| dish_liked| cuisines|approx_cost| reviews_list|menu_item|listed_in_type|listed_in_city|ratings|CuisinesCount|dish_liked_Count|
+--------------------+--------------------+---------------+------------+----------+-----+-----+--------------------+------------+-------------------+--------------------+--------------------+-----------+--------------------+---------+--------------+--------------+-------+-------------+----------------+
https://www.zomat...|942, 21st Main Ro...| Jalsa| Yes| Yes|4.1/5| 775|080 42297555
+91...|Banashankari| Casual Dining|Pasta, Lunch Buff...|North Indian, Mug...| 800|[('Rated 4.0', 'R...| []| Buffet| Banashankari| 4.1| 3| 7|
https://www.zomat...|2nd Floor, 80 Fee...| Spice Elephant| Yes| No|4.1/5| 787| 080 41714161|Banashankari| Casual Dining|Momos, Lunch Buff...|Chinese, North In...| 800|[('Rated 4.0', 'R...| []| Buffet| Banashankari| 4.1| 3| 7|
https://www.zomat...|1112, Next to KIM...|San Churro Cafe| Yes| No|3.8/5| 918| +91 9663487993|Banashankari|Cafe, Casual Dining|Churros, Cannello...|Cafe, Mexican, It...| 800|[('Rated 3.0', "R...| []| Buffet| Banashankari| 3.8| 3| 7|
https://www.zomat...|10, 3rd Floor, La...| Grand Village| No| No|3.8/5| 166|+91 8026612447
+...|Basavanagudi| Casual Dining| Panipuri, Gol Gappe|North Indian, Raj...| 600|[('Rated 4.0', 'R...| []| Buffet| Banashankari| 3.8| 2| 2|
https://www.zomat...|37, 5-1, 4th Floo...|Timepass Dinner| Yes| No|3.8/5| 286|+91 9980040002
+...|Basavanagudi| Casual Dining|Onion Rings, Past...| North Indian| 600|[('Rated 3.0', 'R...| []| Buffet| Banashankari| 3.8| 1| 7|
+--------------------+--------------------+---------------+------------+----------+-----+-----+--------------------+------------+-------------------+--------------------+--------------------+-----------+--------------------+---------+--------------+--------------+-------+-------------+----------------+
only showing top 5 rows

In [16]:
df1.printSchema()


root
-- url: string (nullable = true)
-- address: string (nullable = true)
-- name: string (nullable = true)
-- online_order: string (nullable = true)
-- book_table: string (nullable = true)
-- rate: string (nullable = true)
-- votes: long (nullable = true)
-- phone: string (nullable = true)
-- location: string (nullable = true)
-- rest_type: string (nullable = true)
-- dish_liked: string (nullable = true)
-- cuisines: string (nullable = true)
-- approx_cost: integer (nullable = true)
-- reviews_list: string (nullable = true)
-- menu_item: string (nullable = true)
-- listed_in_type: string (nullable = true)
-- listed_in_city: string (nullable = true)
-- ratings: float (nullable = true)
-- CuisinesCount: integer (nullable = false)
-- dish_liked_Count: integer (nullable = false)

In [17]:
# Creating a view zomato, with the selected columns as features.
df1.select([ 'name','online_order', 'book_table', 'votes', 'location', 'rest_type', 'cuisines', 'approx_cost', 'listed_in_type','ratings','CuisinesCount','dish_liked_Count','listed_in_city']).createOrReplaceTempView('zomato')

In [18]:
df1.count()

Out[17]: 15084

#### Analysis

#### 1. Does online order availability in the restaurants decide the ratings?

In [21]:
%sql
select online_order,avg(ratings) as average_rating from zomato group by online_order 


online_order,average_rating
No,4.094441835752795
Yes,3.977004994571507


##### From the above two averages, we see that they both are almost equal. So we can say that online order does not play an important role in deciding the ratings of the restaurants. Observing closely, we see that people coming to restaurants give slightly higher ratings than those who order online.

#### 2. Does online table booking option availability in the restaurants decide the ratings?

In [24]:
%sql
select book_table,avg(ratings) as average_rating from zomato group by book_table 

book_table,average_rating
No,3.9090531950086946
Yes,4.190330311785239


##### From the above two averages, we see that there is almost 0.3 rating higher for the restaurants with online table booking option availability.

#### 3a. Top five locations for restaurants to avail better ratings

In [27]:
%sql
select location,avg(ratings) as average_rating from zomato group by location order by average_rating desc limit 5 

location,average_rating
City Market,4.399999936421712
Lavelle Road,4.294812734944676
Hennur,4.235294173745548
Koramangala 5th Block,4.218389154924976
Vasanth Nagar,4.2156862230861885


##### The above five locations are the most favourable for restaurants to come up with better customer satisfaction.

#### 3b. Top five restaurants having better ratings

In [30]:
%sql
select name,avg(ratings) as average_rating from zomato group by name order by average_rating desc limit 5 

name,average_rating
Asia Kitchen By Mainland China,4.900000095367432
Byg Brewski Brewing Company,4.900000095367432
SantÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Spa Cuisine,4.900000095367432
Punjab Grill,4.871428694043841
Belgian Waffle Factory,4.844827734190842


##### The above listed five are the restaurants to come up with better customer satisfaction.

#### 3c. Are the top five rated restaurants located in above listed top 5 favourable locations with better customer satisfaction?

In [33]:
%sql
select name,avg(ratings) as avg_rating,location from zomato group by name,location order by avg_rating desc limit 5

name,avg_rating,location
Flechazo,4.900000095367432,Whitefield
Belgian Waffle Factory,4.900000095367432,Brigade Road
Byg Brewski Brewing Company,4.900000095367432,Sarjapur Road
Asia Kitchen By Mainland China,4.900000095367432,Koramangala 5th Block
Milano Ice Cream,4.900000095367432,Indiranagar


##### The above result gives some interesting findings. The top five rated restaurants are not in the top five favourable locations. So location is not the only factor that decides the ratings.

#### 4. Will the number of cuisines have an impact on the customer satisfaction rate (Are customers satisfied with restaurants concentrating on single cuisine or inclined towards multi-cuisines)?

In [36]:
%sql

select CuisinesCount, avg(ratings) as average_rating from zomato group by CuisinesCount order by average_rating desc

CuisinesCount,average_rating
6,4.172398172352649
7,4.1575439034846795
5,4.104934194958524
4,4.046109984178498
-1,4.0
3,3.9934663217263924
2,3.967367345019812
8,3.965833322207133
1,3.9647505932352605


##### From the above analysis, we can say that muti-cuisine restaurants are more liked comapred to single-cuisine restaurants. 

It is interesting to know that, to start a multi-cuisine restaurants it is ideal to have 5 or 6 cuisines. However, if planning for other number of cuisines, the rating is almost same for 1 to 4 cuisines. So, its better to go with single cuisine.

#### Model predictions

##### Because our dependent variable is a continuous varible ranging from zero to five, we cannot use models like logistic regression here. For such continuous varibles with number of inputs, it will be a good example of a regression model problem.

In [40]:
# Selecting the dependent and the independent variables that are identified as most useful attributes to estimate duration

data=df1.select(['online_order','book_table','dish_liked_count','CuisinesCount','listed_in_city','listed_in_type','approx_cost','ratings'])

In [41]:
data.count()

Out[19]: 15084

In [42]:
data=data.dropna()

In [43]:
data.count()

Out[21]: 9977

In [44]:
# Create a 70-30 train test split

train_data,test_data=data.randomSplit([0.7,0.3])

#### Building Random Forest Model:

In [46]:
data.describe().show()

+-------+------------+----------+-----------------+------------------+--------------+--------------+------------------+------------------+
summary|online_order|book_table| dish_liked_count| CuisinesCount|listed_in_city|listed_in_type| approx_cost| ratings|
+-------+------------+----------+-----------------+------------------+--------------+--------------+------------------+------------------+
 count| 9977| 9977| 9977| 9977| 9977| 9977| 9977| 9977|
 mean| null| null|6.114563496040894| 2.915806354615616| null| null| 551.8893454946377| 3.920106237489307|
 stddev| null| null|1.839490781604514|1.4126986429105632| null| null|193.52869719935035|0.4119830337048238|
 min| No| No| -1| 1| BTM| Buffet| 40| 2.0|
 max| Yes| Yes| 7| 8| Whitefield| Pubs and bars| 950| 4.9|
+-------+------------+----------+-----------------+------------------+--------------+--------------+------------------+------------------+

In [47]:
#Created string indexes to a categorical variable in order to convert each category into a specific index(neumerical value)
from pyspark.ml.feature import StringIndexer
online_order_indexer = StringIndexer(inputCol='online_order',outputCol='online_order_index',handleInvalid='keep')
online_order_indexer_df = online_order_indexer.fit(data).transform(data)
online_order_indexer_df.show()

+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+
online_order|book_table|dish_liked_count|CuisinesCount|listed_in_city|listed_in_type|approx_cost|ratings|online_order_index|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+
 Yes| Yes| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 3.8| 0.0|
 No| No| 2| 2| Banashankari| Buffet| 600| 3.8| 1.0|
 Yes| No| 7| 1| Banashankari| Buffet| 600| 3.8| 0.0|
 Yes| Yes| 7| 3| Banashankari| Cafes| 600| 4.6| 0.0|
 Yes| No| 7| 3| Banashankari| Cafes| 700| 4.0| 0.0|
 Yes| No| 7| 5| Banashankari| Cafes| 550| 4.2| 0.0|
 Yes| No| 7| 1| Banashankari| Cafes| 500| 4.1| 0.0|
 Yes| Yes| 7| 4| Banashankari| Cafes| 500| 4.2| 0.0|
 No| No| 7| 2| Banashankari| Cafes| 450| 4.0| 1.0|
 Yes| No| 7| 3| Banashankari| Cafes| 800| 3.8| 0.0|
 Yes| No| 2| 4| Banashankari| Cafes| 500| 3.7| 0.0|
 Yes| No| 7| 2| Banashankari| Cafes| 600| 4.0| 0.0|
 Yes| No| 7| 4| Banashankari| Delivery| 500| 2.8| 0.0|
 Yes| No| 5| 1| Banashankari| Delivery| 750| 3.9| 0.0|
 Yes| No| 7| 3| Banashankari| Delivery| 500| 4.2| 0.0|
 Yes| No| 7| 1| Banashankari| Delivery| 650| 3.9| 0.0|
 Yes| No| 7| 3| Banashankari| Delivery| 600| 4.2| 0.0|
 Yes| No| 7| 4| Banashankari| Delivery| 750| 4.4| 0.0|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+
only showing top 20 rows

In [48]:
#Created string indexes to a categorical variable in order to convert each category into a specific index(neumerical value)
from pyspark.ml.feature import StringIndexer
book_table_indexer = StringIndexer(inputCol='book_table',outputCol='book_table_index',handleInvalid='keep')
book_table_indexer_df = book_table_indexer.fit(online_order_indexer_df).transform(online_order_indexer_df)
book_table_indexer_df.show()

+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+
online_order|book_table|dish_liked_count|CuisinesCount|listed_in_city|listed_in_type|approx_cost|ratings|online_order_index|book_table_index|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+
 Yes| Yes| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 1.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 0.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 3.8| 0.0| 0.0|
 No| No| 2| 2| Banashankari| Buffet| 600| 3.8| 1.0| 0.0|
 Yes| No| 7| 1| Banashankari| Buffet| 600| 3.8| 0.0| 0.0|
 Yes| Yes| 7| 3| Banashankari| Cafes| 600| 4.6| 0.0| 1.0|
 Yes| No| 7| 3| Banashankari| Cafes| 700| 4.0| 0.0| 0.0|
 Yes| No| 7| 5| Banashankari| Cafes| 550| 4.2| 0.0| 0.0|
 Yes| No| 7| 1| Banashankari| Cafes| 500| 4.1| 0.0| 0.0|
 Yes| Yes| 7| 4| Banashankari| Cafes| 500| 4.2| 0.0| 1.0|
 No| No| 7| 2| Banashankari| Cafes| 450| 4.0| 1.0| 0.0|
 Yes| No| 7| 3| Banashankari| Cafes| 800| 3.8| 0.0| 0.0|
 Yes| No| 2| 4| Banashankari| Cafes| 500| 3.7| 0.0| 0.0|
 Yes| No| 7| 2| Banashankari| Cafes| 600| 4.0| 0.0| 0.0|
 Yes| No| 7| 4| Banashankari| Delivery| 500| 2.8| 0.0| 0.0|
 Yes| No| 5| 1| Banashankari| Delivery| 750| 3.9| 0.0| 0.0|
 Yes| No| 7| 3| Banashankari| Delivery| 500| 4.2| 0.0| 0.0|
 Yes| No| 7| 1| Banashankari| Delivery| 650| 3.9| 0.0| 0.0|
 Yes| No| 7| 3| Banashankari| Delivery| 600| 4.2| 0.0| 0.0|
 Yes| No| 7| 4| Banashankari| Delivery| 750| 4.4| 0.0| 0.0|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+
only showing top 20 rows

In [49]:
#Created string indexes to a categorical variable in order to convert each category into a specific index(neumerical value)
from pyspark.ml.feature import StringIndexer
listed_in_city_indexer = StringIndexer(inputCol='listed_in_city',outputCol='listed_in_city_index',handleInvalid='keep')
listed_in_city_indexer_df = listed_in_city_indexer.fit(book_table_indexer_df).transform(book_table_indexer_df)
listed_in_city_indexer_df.show()

+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+
online_order|book_table|dish_liked_count|CuisinesCount|listed_in_city|listed_in_type|approx_cost|ratings|online_order_index|book_table_index|listed_in_city_index|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+
 Yes| Yes| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 1.0| 23.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 0.0| 23.0|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 3.8| 0.0| 0.0| 23.0|
 No| No| 2| 2| Banashankari| Buffet| 600| 3.8| 1.0| 0.0| 23.0|
 Yes| No| 7| 1| Banashankari| Buffet| 600| 3.8| 0.0| 0.0| 23.0|
 Yes| Yes| 7| 3| Banashankari| Cafes| 600| 4.6| 0.0| 1.0| 23.0|
 Yes| No| 7| 3| Banashankari| Cafes| 700| 4.0| 0.0| 0.0| 23.0|
 Yes| No| 7| 5| Banashankari| Cafes| 550| 4.2| 0.0| 0.0| 23.0|
 Yes| No| 7| 1| Banashankari| Cafes| 500| 4.1| 0.0| 0.0| 23.0|
 Yes| Yes| 7| 4| Banashankari| Cafes| 500| 4.2| 0.0| 1.0| 23.0|
 No| No| 7| 2| Banashankari| Cafes| 450| 4.0| 1.0| 0.0| 23.0|
 Yes| No| 7| 3| Banashankari| Cafes| 800| 3.8| 0.0| 0.0| 23.0|
 Yes| No| 2| 4| Banashankari| Cafes| 500| 3.7| 0.0| 0.0| 23.0|
 Yes| No| 7| 2| Banashankari| Cafes| 600| 4.0| 0.0| 0.0| 23.0|
 Yes| No| 7| 4| Banashankari| Delivery| 500| 2.8| 0.0| 0.0| 23.0|
 Yes| No| 5| 1| Banashankari| Delivery| 750| 3.9| 0.0| 0.0| 23.0|
 Yes| No| 7| 3| Banashankari| Delivery| 500| 4.2| 0.0| 0.0| 23.0|
 Yes| No| 7| 1| Banashankari| Delivery| 650| 3.9| 0.0| 0.0| 23.0|
 Yes| No| 7| 3| Banashankari| Delivery| 600| 4.2| 0.0| 0.0| 23.0|
 Yes| No| 7| 4| Banashankari| Delivery| 750| 4.4| 0.0| 0.0| 23.0|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+
only showing top 20 rows

In [50]:
# Vector assembler is used to create a vector of input features

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['online_order_index','book_table_index','listed_in_city_index',
                                       'dish_liked_count','CuisinesCount','approx_cost'],
                            outputCol="features")
output = assembler.transform(listed_in_city_indexer_df)
output.show(2)

+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+--------------------+
online_order|book_table|dish_liked_count|CuisinesCount|listed_in_city|listed_in_type|approx_cost|ratings|online_order_index|book_table_index|listed_in_city_index| features|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+--------------------+
 Yes| Yes| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 1.0| 23.0|[0.0,1.0,23.0,7.0...|
 Yes| No| 7| 3| Banashankari| Buffet| 800| 4.1| 0.0| 0.0| 23.0|[0.0,0.0,23.0,7.0...|
+------------+----------+----------------+-------------+--------------+--------------+-----------+-------+------------------+----------------+--------------------+--------------------+
only showing top 2 rows

In [51]:
#Applying features and lables to provide the same to model.
df_classifier = output.selectExpr("features as features","ratings as label")
df_classifier.show(2)

+--------------------+-----+
 features|label|
+--------------------+-----+
[0.0,1.0,23.0,7.0...| 4.1|
[0.0,0.0,23.0,7.0...| 4.1|
+--------------------+-----+
only showing top 2 rows

In [52]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
#from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.



featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=5).fit(df_classifier)

In [53]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df_classifier.randomSplit([0.7, 0.3])

In [54]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

In [55]:
# Chain indexer and forest in a Pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

In [56]:
# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [57]:
predictions = model.transform(testData)

In [58]:
# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary only

+------------------+-----+--------------------+
 prediction|label| features|
+------------------+-----+--------------------+
3.7280129426551896| 4.0|[0.0,0.0,9.0,-1.0...|
3.7298808848009513| 3.8|[0.0,0.0,9.0,1.0,...|
3.7298808848009513| 3.8|[0.0,0.0,9.0,1.0,...|
3.7110295141413916| 3.9|[0.0,0.0,9.0,1.0,...|
 3.691252164640005| 3.7|[0.0,0.0,9.0,2.0,...|
+------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.366692
RandomForestRegressionModel (uid=RandomForestRegressor_51dd37a99c75) with 20 trees

##### For the Random forest regression model, the Root Mean Squared Error 0.37 explains that the standard deviation of the residuals is low. It also indicates that most of the data points lie closely to the regression line.

### Decision Tree:

In [61]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [62]:
# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

In [63]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)


In [64]:
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+------------------+-----+--------------------+
 prediction|label| features|
+------------------+-----+--------------------+
 3.707476642644294| 4.0|[0.0,0.0,9.0,-1.0...|
 3.707476642644294| 3.8|[0.0,0.0,9.0,1.0,...|
 3.707476642644294| 3.8|[0.0,0.0,9.0,1.0,...|
3.5490566165942066| 3.9|[0.0,0.0,9.0,1.0,...|
3.5490566165942066| 3.7|[0.0,0.0,9.0,2.0,...|
+------------------+-----+--------------------+
only showing top 5 rows

In [65]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
treeModel = model.stages[1]
# summary only
print(treeModel)

Root Mean Squared Error (RMSE) on test data = 0.374448
DecisionTreeRegressionModel (uid=DecisionTreeRegressor_f72f0852cbb3) of depth 5 with 61 nodes

##### For the Decision Tree model, the Root Mean Squared Error 0.377 explains that the standard deviation of the residuals is low. It also indicates that most of the data points lie closely to the regression line.

### Linear Regression

In [68]:
from pyspark.ml.regression import LinearRegression

In [69]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(trainingData)

In [70]:
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0]
Intercept: 3.918657033211058

In [71]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

numIterations: 1
objectiveHistory: [0.4999999999999929]
+--------------------+
 residuals|
+--------------------+
-0.01865693784362632|
-0.41865703321105796|
-0.41865703321105796|
 -0.8186571285784896|
-0.21865698552734214|
-0.21865698552734214|
 -0.7186569855273421|
-0.21865698552734214|
-0.21865698552734214|
-0.11865708089477378|
-0.01865693784362632|
 0.08134296678894204|
-0.21865698552734214|
 -0.918657033211058|
 -0.918657033211058|
 0.08134296678894204|
-0.11865708089477378|
 -0.3186571285784896|
 -0.3186571285784896|
 -0.3186571285784896|
+--------------------+
only showing top 20 rows

RMSE: 0.413286
r2: 0.000000

##### For the Linear regression model, the Root Mean Squared Error 0.413 explains that the standard deviation of the residuals is low. It also indicates that most of the data points lie closely to the regression line.

##### From the above three models evaluations, we can see that the Root Mean Square Error of these models are almost same, as we have choosen the best possible models for this kind of data. On closely observing the model performance, we see that Random Forest model performed better than other two models with lower RMSE.

##### As a consulting firm, we are now in a position to provide a deep analysis on which location, number of cusinies, price range and theme of restaurent to choose to any prospective client who wants to get into the restaurant business in city of Banglore. Our model is good enough to predict the ratings which gives an idea of how the business performed based on various factors.